# Exploratory Data Analysis

## Things done:
- Factor Distribution
- Missing %
- Correlation Analysis for Numerical factors
- Target summary for categorical factors
- Feature Importance using Random Forest 

### Dataset Details

100,000 insurance records for different persons with demographic and medical data. Total 54 columns are divided in following groups

1. Demographics & Socioeconomic:
person_id, age, sex, region, urban_rural, income, education, marital_status, employment_status, household_size, dependents

2. Lifestyle & Habits:
bmi, smoker, alcohol_freq, exercise_frequency, sleep_hours, stress_level

3. Health & Clinical:
hypertension, diabetes, copd, cardiovascular, cancer_history, kidney_disease, liver_disease, arthritis, mental_health, chronic_count, systolic_bp, diastolic_bp, ldl, hba1c, risk_score, is_high_risk

4. Healthcare Utilization & Procedures:
visits_last_year, hospitalizations_last_3yrs, days_hospitalized_last_3yrs, medication_count, proc_imaging, proc_surgery, proc_psycho, proc_consult_count, proc_lab, had_major

5. Insurance & Policy:
plan_type, network_tier, deductible, copay, policy_term_years, policy_changes_last_2yrs, provider_quality

6. Medical Costs & Claims:
annual_medical_cost, annual_premium, monthly_premium, claims_count, avg_claim_amount, total_claims_paid


In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from utils import get_config_params

In [4]:
# GET CONFIG PARAMS
input_data_path = get_config_params('00_config.yaml')
print(input_data_path)

# LOAD DATA
data = pd.read_csv(input_data_path)

print(data.shape)
data.head()


/Users/vss/.cache/kagglehub/datasets/mohankrishnathalla/medical-insurance-cost-prediction/versions/1/medical_insurance.csv
(100000, 54)


,person_id,age,sex,region,urban_rural,income,education,marital_status,employment_status,household_size,...,liver_disease,arthritis,mental_health,proc_imaging_count,proc_surgery_count,proc_physio_count,proc_consult_count,proc_lab_count,is_high_risk,had_major_procedure
0,75722,52,Female,North,Suburban,22700.0,Doctorate,Married,Retired,3,...,0,1,0,1,0,2,0,1,0,0
1,80185,79,Female,North,Urban,12800.0,No HS,Married,Employed,3,...,0,1,1,0,0,1,0,1,1,0
2,19865,68,Male,North,Rural,40700.0,HS,Married,Retired,5,...,0,0,1,1,0,2,1,0,1,0
3,76700,15,Male,North,Suburban,15600.0,Some College,Married,Self-employed,5,...,0,0,0,1,0,0,1,0,0,0
4,92992,53,Male,Central,Suburban,89600.0,Doctorate,Married,Self-employed,2,...,0,1,0,2,0,1,1,0,1,0


## Dataset Overview and Quality Assessments

In [22]:
# Eeach row is record for an individual person
print(data['person_id'].nunique() == data['person_id'].shape[0])
print()

# Distribution of numeric and categorical columns
print(data.dtypes.value_counts())
print()

# Distribution of target: Highly right skewed data
target = 'annual_medical_cost'
print("Distribution of target")
print(data[target].describe())



True

int64      31
float64    13
object     10
Name: count, dtype: int64

Distribution of target
count    100000.000000
mean       3009.451907
std        3127.462822
min          55.550000
25%        1175.117500
50%        2082.575000
75%        3707.957500
max       65724.900000
Name: annual_medical_cost, dtype: float64


In [ ]:
# GENERATE YDATA PROFILING REPORT
#from ydata_profiling import ProfileReport
#profile = ProfileReport(data, title="Medical Insurance Cost Dataset Profiling Report", explorative=True)
#profile.to_file("../data/output/profile_report.html")
#rint(f"Saved profiling report to: {report_path}")


Export report to file: 100%|██████████| 1/1 [00:00<00:00, 38.99it/s]

Saved profiling report to: ../data/output/profile_report.html


## Missing Values
Only one columns is missing, all other columns is having 100% availability

In [82]:
columns_missing = data.isna().sum()
print(columns_missing[columns_missing >0])

alcohol_freq    30083
dtype: int64


## Distribution of Factors

In [90]:
cat_columns = data.select_dtypes(exclude=[np.number]).columns.to_list()
for col in cat_columns:
    print("\n", col)
    print(data[col].value_counts())


 sex
sex
Female    49193
Male      48794
Other      2013
Name: count, dtype: int64

 region
region
South      28029
North      22027
East       19984
West       17879
Central    12081
Name: count, dtype: int64

 urban_rural
urban_rural
Urban       60019
Suburban    25021
Rural       14960
Name: count, dtype: int64

 education
education
Bachelors       27996
Some College    25112
HS              24827
Masters         13987
No HS            5120
Doctorate        2958
Name: count, dtype: int64

 marital_status
marital_status
Married     53252
Single      35715
Divorced     6984
Widowed      4049
Name: count, dtype: int64

 employment_status
employment_status
Employed         55269
Retired          19864
Unemployed       12939
Self-employed    11928
Name: count, dtype: int64

 smoker
smoker
Never      69709
Former     18163
Current    12128
Name: count, dtype: int64

 alcohol_freq
alcohol_freq
Occasional    45078
Weekly        19833
Daily          5006
Name: count, dtype: int64

 plan_typ

In [ ]:
data.select_dtypes(include=[np.number]).describe().T

,count,mean,std,min,25%,50%,75%,max
person_id,100000.0,50000.500000,28867.657797,1.00,25000.7500,50000.5000,75000.2500,100000.00
age,100000.0,47.521500,15.988752,0.00,37.0000,48.0000,58.0000,100.00
income,100000.0,49873.905000,46800.214127,1100.00,21100.0000,36200.0000,62200.0000,1061800.00
household_size,100000.0,2.430900,1.075126,1.00,2.0000,2.0000,3.0000,9.00
dependents,100000.0,0.898380,0.950654,0.00,0.0000,1.0000,1.0000,7.00
bmi,100000.0,26.990512,4.994883,12.00,23.6000,27.0000,30.4000,50.40
visits_last_year,100000.0,1.927650,1.737730,0.00,1.0000,2.0000,3.0000,25.00
hospitalizations_last_3yrs,100000.0,0.093640,0.304848,0.00,0.0000,0.0000,0.0000,3.00
days_hospitalized_last_3yrs,100000.0,0.373350,1.373011,0.00,0.0000,0.0000,0.0000,21.00
medication_count,100000.0,1.236320,1.209358,0.00,0.0000,1.0000,2.0000,11.00


## Numerical Factors: Correlation Analysis

top correlated factors with target 'annual medical cost'
- monthly_premium	                0.965
- annual_premium	                0.965
- total_claims_paid	                0.739
- avg_claim_amount	                0.633
- risk_score	                    0.306
- chronic_count	                    0.297
- is_high_risk	                    0.252
- days_hospitalized_last_3yrs	    0.230
- hospitalizations_last_3yrs	    0.209

Premiums are set before claims are filled, so seem like there may not a target leakage, and totally good to use in input models. But I think premiums are estimated by any other model or human, we can drop them and estiamte cost purely from user demographics, it's health metrics and past claims.

total_claims_paid and avg_claim_amount seem like factors that may be derived from cost, not sure

In [37]:
target = 'annual_medical_cost'
numeric_features = data.select_dtypes(exclude=['object']).columns
corr_df = data[numeric_features].corr()

target_corr_df = corr_df['annual_medical_cost'].sort_values(ascending=False)
# ignore self-correlation
np.fill_diagonal(corr.values, np.nan)

top5 = (
    corr.stack()
        .groupby(level=0)
        .apply(lambda s: s.loc[s.abs().sort_values(ascending=False).index[:5]])
        .reset_index()
        .rename(columns={'level_1': 'variable', 'level_2': 'with', 0: 'correlation'})
)

with pd.ExcelWriter('../data/output/Correlation_Analysis(NumericFactors).xlsx') as writer:
     target_corr_df.reset_index().to_excel(writer, sheet_name='target correlation', index=False)
     top5.to_excel(writer, sheet_name='top correlation factors', index=False)
     corr_df.to_excel(writer, sheet_name='correlation matrix')

## Categorical Factors Analysis
Based on mean for the feature=value, 
Most important factors seems
- Flags that indicates disease i.e. liver_disease, hypertension 
- proc_consult_count: denotes procedure/consultation count. Should we discard it, as it can have target leakage? Same goes with other "proc" factors i.e proc_consult_count, proc_imaging_count, proc_physio_count, proc_surgery_count, proc_lab_count, medication_count
- is_high_risk
- past health information i.e. visits_last_year, days_hospitalized_last_3yrs
Least Importanta factors seems
- Demographics features i.e. sex, household_size, dependents, education (age seems important, middle age person tend to have higher cost)

In [45]:
cat_features = list(set(data.select_dtypes(include=['object', 'int']).columns).difference(
    ['person_id', 'age', 'claims_count']
))

for feature in cat_features:
    print()
    print(feature)
    print(data.groupby(feature).agg({target: 'mean', 'person_id': 'count'}))
    


copay
       annual_medical_cost  person_id
copay                                
10             3005.605728      40059
20             3004.597654      34916
30             3013.774142      19906
50             3055.852922       5119

education
              annual_medical_cost  person_id
education                                   
Bachelors             2983.472127      27996
Doctorate             2913.390791       2958
HS                    3014.149219      24827
Masters               3044.756974      13987
No HS                 3080.779098       5120
Some College          3010.879565      25112

proc_consult_count
                    annual_medical_cost  person_id
proc_consult_count                                
0                           2942.283724      61668
1                           3008.140681      28389
2                           3253.391880       7812
3                           3917.179104       1696
4                           4526.358599        364
5                

## Data Preprocessing

In [68]:
# Factors that we going to deal with
target='annual_medical_cost'
factors = ['age', 'sex', 'region', 'urban_rural', 'income',
       'education', 'marital_status', 'employment_status', 'household_size',
       'dependents', 'bmi', 'smoker', 'alcohol_freq', 'visits_last_year',
       'hospitalizations_last_3yrs', 'days_hospitalized_last_3yrs',
        'systolic_bp', 'diastolic_bp', 'ldl', 'hba1c',
       'chronic_count', 'hypertension', 'diabetes', 'asthma', 'copd',
       'cardiovascular_disease', 'cancer_history', 'kidney_disease',
       'liver_disease', 'arthritis', 'mental_health', 'is_high_risk', 'had_major_procedure']

In [69]:
# one hot encode the categorical/object columns
data2 = data[[target] + factors]
num_featuers = list(data2.select_dtypes(include=['int', 'float']).columns)
cat_features = list(data2.select_dtypes(include=['object']).columns)

# 
data3 = pd.get_dummies(data2, drop_first=True)
print(data2.shape)
print(data3.shape)

(100000, 34)
(100000, 49)


## Feature Importance

## 

In [77]:
from sklearn.ensemble import RandomForestRegressor

model_forest = RandomForestRegressor(n_estimators=100, min_samples_leaf=100, max_depth=15)

y = data3[target].values
X = data3.drop([target] , axis=1)
model_forest.fit(X, y)


,n_estimators,100
,criterion,'squared_error'
,max_depth,15
,min_samples_split,2
,min_samples_leaf,100
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [80]:
feature_importance_df = pd.DataFrame({
    'features': X.columns,
    'importance': model_forest.feature_importances_
}).sort_values('importance', ascending=False)


with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(feature_importance_df)

                           features  importance
23                     is_high_risk    0.285007
7       days_hospitalized_last_3yrs    0.230074
12                    chronic_count    0.181389
45                     smoker_Never    0.066323
0                               age    0.038882
5                  visits_last_year    0.028867
44                    smoker_Former    0.028540
4                               bmi    0.026498
10                              ldl    0.021134
1                            income    0.018570
11                            hba1c    0.017568
8                       systolic_bp    0.014009
9                      diastolic_bp    0.011765
6        hospitalizations_last_3yrs    0.002887
2                    household_size    0.002652
32                urban_rural_Urban    0.002545
25                         sex_Male    0.002533
38           marital_status_Married    0.002364
46          alcohol_freq_Occasional    0.002232
3                        dependents    0